# Árvore Patrícia

**Apresentação para o seminário da disciplina Projeto de Algoritmos II**

## Integrantes

- Eduardo Gil Serrão Cardoso
- Gabriela Souza Maximino
- Igor Matheus Souza Moreira
- Marco Aurélio Lima do Nascimento Júnior

## Agenda

- [Introdução](#Introdução)
    - [Pesquisa Digital](#Pesquisa-Digital)
    - [Árvore PATRICIA](#Árvore-PATRICIA)
- [Definições e propriedades]()
- [Operações e complexidade]()
- [Comparação com outras estruturas de dados]()
- [Aplicações]()
- [Conclusões]()
- [Referências]()

## Introdução

- Tem-se uma maior capacidade de armazenamento em dispositivos de memória (primária e secundária);
- Arquivos de grande conteúdo (que estão na RAM) puderam ser manipulados;


- **Problema:** como recuperar uma informação em meio a tantas dentro desse arquivo?
- **Solução:** métodos de pesquisa: sequencial, binária… e **pesquisa digital!**

### Pesquisa Digital

- Arquivos são divididos em registros. Cada registro possui uma chave;
- O **objetivo das pesquisas** em geral é encontrar, baseado em uma chave de busca, chaves iguais no arquivo;
- A busca digital consiste em representar essas chaves como dígitos, podendo ser divisíveis;
- A estrutura básica para uma pesquisa digital é a árvore digital;
- As árvores digitais também são conhecidas como Trie (re**TRIE**val);
- A **árvore PATRICIA** é uma árvore digital!

### Árvore PATRICIA

- Acrônimo para ***P*** *ractical* ***A*** *lgorithm* ***t*** *o* ***R*** *etrieve* ***I*** *nformation* ***C*** *oded* ***i*** *n* ***A*** *lphanumeric* (Algoritmo Prático para Obter Informação Codificada em Alfanumérico);
- Criada por Donald R. Morrison em 1968;
- Implementação eficiente da árvore digital binária.

Morrison, D. R. (1968). PATRICIA—practical algorithm to retrieve information coded in alphanumeric. Journal of the ACM (JACM), 15(4), 514-534.

[Voltar ao topo](#Árvore-Patrícia)

## Definições e Propriedades

Usaremos as sete palavras **casco**, **cascalho**, **cascata**, **dama**, **domando**, **dominar** e **domínio** para representar as árvores.

- **Árvore Trie:** cada nível representa um caractere
    - **Problema:** gera muitos nós.
    
![](imagens/exemplo_de_trie.jpg)
    
- **Árvore Radix:** árvore de prefixos.
    - **Melhoria:** diminui o número de nós criados.
    
![](imagens/exemplo_de_radix.jpg)
    
- **Árvore Patrícia:** uma árvore Trie compacta... em algumas definições.

![](imagens/exemplo_de_patricia.png)